In [1]:
import re
from operator import add

# from mrjob.job import MRJob
WORD_RE = re.compile(r"[\w']+")

def get_words(line):
    return WORD_RE.findall(line)

In [2]:
input_path = "small_input.txt"

In [15]:
# Spark may not be available where script is launched
from pyspark import SparkContext
sc = SparkContext(appName='mrjob Spark wordcount script')

In [16]:
lines = sc.textFile(input_path)
lines.collect()

['1|2,3|0|GRAY',
 '2|1,3,4,5|9999|WHITE',
 '3|1,2,4|9999|WHITE',
 '4|2,3|9999|WHITE',
 '5|2|9999|WHITE']

In [17]:
class Node:
    def __init__(self):
        self.characterID = ''
        self.connections = []
        self.distance = 9999
        self.color = 'WHITE'

    #Format is ID|EDGES|DISTANCE|COLOR
    def fromLine(self, line):
        fields = line.split('|')
        if (len(fields) == 4):
            self.characterID = fields[0]
            self.connections = fields[1].split(',')
            if len(self.connections) == 1 and self.connections[0] == '':
                self.connections = []
            self.distance = int(fields[2])
            self.color = fields[3].rstrip('\t')

    def getLine(self):
        connections = ','.join(self.connections)
        return '|'.join( (self.characterID, connections, str(self.distance), self.color) )

In [25]:
def f_map(line):
    node = Node()
    node.fromLine(line)

    if (node.color == 'WHITE'): 
        counterName = ("Number of remaining nodes")
        # self.increment_counter('remained', counterName, 1)
    #If this node needs to be expanded...
    if (node.color == 'GRAY'):
        for connection in node.connections:
            vnode = Node()
            vnode.characterID = connection
            vnode.distance = int(node.distance) + 1
            vnode.color = 'GRAY'
            yield connection, vnode.getLine()

    #We've processed this node, so color it black
        node.color = 'BLACK'        

    #Emit the input node so we don't lose it.
    yield node.characterID, node.getLine()

def f_reduce(key, values):
    edges = []
    distance = 9999
    color = 'WHITE'
    
    # sys.stderr.write("REDUCER INPUT: ({0}, {1})\n".format(key, values))
    
    for value in values:
        node = Node()
        node.fromLine(value)

        if (len(node.connections) > 0):
            edges.extend(node.connections)

        if (node.distance < distance):
            distance = node.distance

        if ( node.color == 'BLACK' ):
            color = 'BLACK'

        if ( node.color == 'GRAY' and color == 'WHITE' ):
            color = 'GRAY'

    node = Node()
    node.characterID = key
    node.distance = distance
    node.color = color
    node.connections = edges

    yield key, node.getLine()

In [26]:
# counts = (
#     lines.flatMap(self.get_words)
#     .map(lambda word: (word, 1))
#     .reduceByKey(add))
map_result = lines.map(f_map)
result = map_result.reduceByKey(f_reduce)
result.collect()
# result.saveAsTextFile(output_path)

sc.stop()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 8) (172.27.201.81 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pastrol/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/pastrol/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/home/pastrol/anaconda3/lib/python3.8/site-packages/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/pastrol/anaconda3/lib/python3.8/site-packages/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, ite